# 💳 Finanzas y Banca: riesgo, fraude, rentabilidad
KPIs: NPL, PD/LGD/EAD, NIM, churn, CAC, fraude por canal.

## Casos de uso
- Scoring de crédito y límites dinámicos
- AML/KYC, detección de anomalías transaccionales
- Segmentación RFM y propensión a compra
- Monitoreo de liquidez y escenarios regulatorios (IFRS9/Basilea)

In [ ]:
# Mini ejemplo PD con features simples (toy)
def prob_default(score: float) -> float:
    import math
    z = -2.0 + 0.03 * score
    return 1/(1+math.exp(-z))

[round(prob_default(s),3) for s in [300,500,700]]

## Arquitectura
- Flujos transaccionales en streaming
- Feature store para variables de riesgo
- Modelos gobernados, explainability y trazabilidad
- Data contracts y auditoría regulatoria

## 📌 Valor y cumplimiento
- Orígenes consolidados y trazables → auditorías y reporting regulatorio.
- Modelos de riesgo consistentes (features versionadas) → decisiones rápidas.
- Prevención de fraude y AML → pérdidas evitadas y reputación.
- Costos optimizados en nube con políticas de datos fríos/calientes.


In [ ]:
# Práctica: PD y Pérdida Esperada (ECL) en toy data
import pandas as pd

clientes = pd.DataFrame({
    'score': [300, 450, 600, 700, 800],
    'EAD': [1000, 2000, 5000, 8000, 12000],  # exposición
    'LGD': [0.6, 0.5, 0.45, 0.4, 0.35],      # pérdida dado default
})

import math
pd_func = lambda s: 1/(1+math.exp(-(-2.0 + 0.003*s)))
clientes['PD'] = clientes['score'].apply(pd_func)
clientes['ECL'] = clientes['PD'] * clientes['LGD'] * clientes['EAD']
print(clientes[['score','PD','EAD','LGD','ECL']])
print('ECL total:', round(clientes['ECL'].sum(), 2))


## 🏗️ Arquitectura y gobierno
- Ingesta transaccional + streaming; particionado por fecha y cliente.
- Feature store para riesgo (ventanas, agregaciones, drifts monitorizados).
- Lineage, versionado de modelos y explainability; alertas de sesgo.
- Controles de acceso por dominio y tokenización de PII.
